In [28]:
import pandas as pd
import numpy as np
import random

# Carregar as matrizes de distância dos arquivos CSV
path_restaurantes = 'C:\\Users\\neumann\\Documents\\projects\\get\\restaurantes_distance_matrix.csv'
path_partidos = 'C:\\Users\\neumann\\Documents\\projects\\get\\partidos_distance_matrix.csv'
path_escolas = 'C:\\Users\\neumann\\Documents\\projects\\get\\escolas_distance_matrix.csv'
path_all = 'C:\\Users\\neumann\\Documents\\projects\\get\\all_distance_matrix.csv'

# Carregar os dados usando pandas
restaurantes_distance_matrix = pd.read_csv(path_restaurantes, index_col=0)
partidos_distance_matrix = pd.read_csv(path_partidos, index_col=0)
escolas_distance_matrix = pd.read_csv(path_escolas, index_col=0)
all_distance_matrix = pd.read_csv(path_all, index_col=0)

# Exibir as matrizes carregadas (opcional)
print("Matriz de Distância - Restaurantes:")
print(restaurantes_distance_matrix.head())
print("\nMatriz de Distância - Partidos:")
print(partidos_distance_matrix.head())
print("\nMatriz de Distância - Escolas:")
print(escolas_distance_matrix.head())
print("\nMatriz de Distância - Todos os Pontos:")
print(all_distance_matrix.head())


Matriz de Distância - Restaurantes:
                       Coco Bambu  Outback Steakhouse  Carneiro do Ordones  \
Nome                                                                         
Coco Bambu               0.000000            3.524717             7.010785   
Outback Steakhouse       3.524717            0.000000             7.118125   
Carneiro do Ordones      7.010785            7.118125             0.000000   
Cervejaria Turatti       1.017932            2.529526             6.679097   
Santa Grelha Iguatemi    3.524717            0.000000             7.118125   

                       Cervejaria Turatti  Santa Grelha Iguatemi  \
Nome                                                               
Coco Bambu                       1.017932               3.524717   
Outback Steakhouse               2.529526               0.000000   
Carneiro do Ordones              6.679097               7.118125   
Cervejaria Turatti               0.000000               2.529526   
Santa Gre

In [29]:
# Obter inputs do usuário
num_restaurantes = int(input("A entrega vai ser em quantos restaurantes? "))
num_partidos = int(input("A entrega vai ser em quantos partidos? "))
num_escolas = int(input("A entrega vai ser em quantas escolas? "))

# Seleção aleatória de pontos
restaurantes_pontos = random.sample(list(restaurantes_distance_matrix.index), num_restaurantes)
partidos_pontos = random.sample(list(partidos_distance_matrix.index), num_partidos)
escolas_pontos = random.sample(list(escolas_distance_matrix.index), num_escolas)

# Combina todos os pontos escolhidos
pontos_escolhidos = restaurantes_pontos + partidos_pontos + escolas_pontos

# Exibir pontos escolhidos
print("\nPontos Escolhidos para Restaurantes:", restaurantes_pontos)
print("Pontos Escolhidos para Partidos:", partidos_pontos)
print("Pontos Escolhidos para Escolas:", escolas_pontos)
print("\nTodos os Pontos Escolhidos:", pontos_escolhidos)



Pontos Escolhidos para Restaurantes: ['Coco Bambu', 'Outback Steakhouse', 'Misaki', 'Cantinho do Frango', 'Cervejaria Turatti', 'Carneiro do Ordones', 'Restaurante Brazão', 'Santa Grelha Iguatemi', 'Chico do Caranguejo', 'Mangue Azul']
Pontos Escolhidos para Partidos: ['Partido dos Trabalhadores Diretório Estadual do Ceará', 'PSDB Partido da Social Democracia Brasileira', 'Partido Liberal - Comitê de Caucaia', 'União Brasil Fortaleza', 'Partido Democrático Trabalhista | PDT Ceará', 'MDB', 'Avante', 'Republicanos Ceará', 'Partido Verde - Ceará', 'Diretório Estadual do Partido Novo Ceará']
Pontos Escolhidos para Escolas: ['Colégio 7 de Setembro', 'Colégio Antares', 'Colégio Master', 'Colégio Christus', 'Colégio Lourenço Filho', 'Colégio Santa Cecília', 'Colégio Santo Tomás de Aquino', 'Colégio Farias Brito Central', 'Colégio Ari de Sá Cavalcante', 'Colégio Provecto']

Todos os Pontos Escolhidos: ['Coco Bambu', 'Outback Steakhouse', 'Misaki', 'Cantinho do Frango', 'Cervejaria Turatti', '

In [38]:
import pandas as pd
import numpy as np
import random

# Identificar o ponto fixo (centro de distribuição) "Servprint"
depot_name = "Servprint"
depot_index = all_distance_matrix.index.get_loc(depot_name)

# Função para calcular a distância total de uma rota
def calculate_total_distance(route, distance_matrix):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i], route[i + 1]]
    return total_distance

# Função para gerar uma solução inicial simples onde todos os pontos são visitados na ordem
def initial_simple_solution(depot, distance_matrix, pontos_indices):
    route = [depot] + pontos_indices + [depot]  # Começa e termina no depósito
    total_distance = calculate_total_distance(route, distance_matrix)
    return route, total_distance

# Função para destruir parcialmente a solução (remove alguns pontos aleatoriamente)
def destroy_route(route, num_remove):
    depot = route[0]
    removed = random.sample(route[1:-1], num_remove)  # Não remove o depósito no início e no fim
    new_route = [depot] + [city for city in route[1:-1] if city not in removed] + [depot]
    return new_route, removed

# Função para reconstruir a rota reinserindo pontos removidos
def reconstruct_route(route, removed, distance_matrix):
    depot = route[0]
    current_route = route[:-1]  # Remove o retorno ao depósito temporariamente
    
    for city in removed:
        best_position = None
        best_cost = float('inf')
        # Tenta inserir a cidade removida em todas as posições possíveis na rota
        for i in range(1, len(current_route)):
            new_route = current_route[:i] + [city] + current_route[i:]
            new_route_distance = calculate_total_distance(new_route + [depot], distance_matrix)
            if new_route_distance < best_cost:
                best_cost = new_route_distance
                best_position = i
        # Insere na melhor posição
        current_route.insert(best_position, city)
    
    current_route.append(depot)  # Reinsere o retorno ao depósito
    total_distance = calculate_total_distance(current_route, distance_matrix)
    return current_route, total_distance

# Função LNS para resolver o VRP
def lns_vrp_classic(depot, distance_matrix, pontos_indices, num_iterations=100, num_remove=3):
    # Solução inicial simples
    best_route, best_distance = initial_simple_solution(depot, distance_matrix, pontos_indices)
    
    for iteration in range(num_iterations):
        # Destruição: Remove alguns pontos da rota
        destroyed_route, removed = destroy_route(best_route, num_remove)
        
        # Reconstrução: Reinsere os pontos removidos
        new_route, new_distance = reconstruct_route(destroyed_route, removed, distance_matrix)
        
        # Verifica se a nova solução é melhor
        if new_distance < best_distance:
            best_route, best_distance = new_route, new_distance
            print(f"Iteração {iteration + 1}: Nova melhor distância encontrada: {best_distance:.2f} km")
    
    return best_route, best_distance

# Preparar os dados da matriz de distância
pontos_escolhidos = list(all_distance_matrix.index)  # Todos os pontos da matriz
pontos_indices = list(range(len(pontos_escolhidos)))  # Todos os índices de pontos

# Excluir o índice do depósito dos índices dos pontos a visitar
pontos_indices.remove(depot_index)

# Converter a matriz de distância para NumPy
distance_matrix_np = all_distance_matrix.values

# Executa o algoritmo LNS ajustado
rota_otimizada, distancia_total = lns_vrp_classic(depot_index, distance_matrix_np, pontos_indices)

# Exibe a rota final e a distância total
rota_nomes = [pontos_escolhidos[index] for index in rota_otimizada]
print("\nRota otimizada LNS (VRP clássico com 'Servprint' como ponto fixo):")
print(" -> ".join(rota_nomes))
print(f"\nDistância total percorrida: {distancia_total:.2f} km")

# Preparar os dados da rota para exportação
rota_data = {
    "Order": list(range(1, len(rota_nomes) + 1)),  # Ordem dos pontos na rota
    "Nome": rota_nomes  # Nomes dos pontos na rota
}

# Criar um DataFrame com os dados da rota
rota_df = pd.DataFrame(rota_data)

# Exportar o DataFrame para um arquivo CSV
output_csv_path = 'rota_otimizada_vrp.csv'
rota_df.to_csv(output_csv_path, index=False)

print(f"Rota otimizada exportada para o arquivo: {output_csv_path}")

Iteração 1: Nova melhor distância encontrada: 156.06 km
Iteração 2: Nova melhor distância encontrada: 144.54 km
Iteração 3: Nova melhor distância encontrada: 139.01 km
Iteração 4: Nova melhor distância encontrada: 138.45 km
Iteração 5: Nova melhor distância encontrada: 133.94 km
Iteração 6: Nova melhor distância encontrada: 125.23 km
Iteração 7: Nova melhor distância encontrada: 108.31 km
Iteração 8: Nova melhor distância encontrada: 100.43 km
Iteração 9: Nova melhor distância encontrada: 99.90 km
Iteração 10: Nova melhor distância encontrada: 97.39 km
Iteração 11: Nova melhor distância encontrada: 92.89 km
Iteração 12: Nova melhor distância encontrada: 92.00 km
Iteração 13: Nova melhor distância encontrada: 90.45 km
Iteração 14: Nova melhor distância encontrada: 88.97 km
Iteração 15: Nova melhor distância encontrada: 88.84 km
Iteração 16: Nova melhor distância encontrada: 88.83 km
Iteração 17: Nova melhor distância encontrada: 86.37 km
Iteração 20: Nova melhor distância encontrada: 85

In [44]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import osmnx as ox
import networkx as nx

# Caminhos para os arquivos
pos_file_path = 'C:\\Users\\neumann\\Documents\\projects\\get\\all_pos.csv'
rota_file_path = 'C:\\Users\\neumann\\Documents\\projects\\get\\rota_otimizada_vrp.csv'

# Carregar a localização dos pontos a partir do arquivo CSV
positions = pd.read_csv(pos_file_path)

# Carregar a rota otimizada a partir do arquivo CSV
rota_otimizada = pd.read_csv(rota_file_path)

# Criar um dicionário para buscar a localização dos pontos pelo nome
positions_dict = positions.set_index('Nome')[['Latitude', 'Longitude']].T.to_dict('dict')

# Extrair os pontos da rota otimizada
rota_pontos = rota_otimizada['Nome'].tolist()

# Coordenadas médias para centralizar o mapa
latitudes = [positions_dict[point]['Latitude'] for point in rota_pontos]
longitudes = [positions_dict[point]['Longitude'] for point in rota_pontos]
map_center = (sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes))

# Função para criar um mapa com rota
def criar_mapa(positions_dict, rota_pontos, nome_arquivo):
    # Aumentar o raio de busca para garantir cobertura da área de Fortaleza e região metropolitana
    radius = 50000  # 50 km
    
    # Baixar a rede viária da área de interesse
    G = ox.graph_from_point(map_center, dist=radius, network_type='drive')
    
    # Criar o mapa centrado na média das coordenadas
    mapa = folium.Map(location=map_center, zoom_start=12)
    marker_cluster = MarkerCluster().add_to(mapa)
    
    # Adicionar marcadores para cada ponto na rota
    for point in rota_pontos:
        folium.Marker(
            location=[positions_dict[point]['Latitude'], positions_dict[point]['Longitude']],
            popup=point,
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(marker_cluster)
    
    # Adicionar linhas conectando os pontos com base na rede viária
    rota_coords = []
    for i in range(len(rota_pontos) - 1):
        start = rota_pontos[i]
        end = rota_pontos[i + 1]
        
        start_coords = (positions_dict[start]['Latitude'], positions_dict[start]['Longitude'])
        end_coords = (positions_dict[end]['Latitude'], positions_dict[end]['Longitude'])
        
        # Encontrar os nós mais próximos na rede de ruas
        start_node = ox.distance.nearest_nodes(G, start_coords[1], start_coords[0])
        end_node = ox.distance.nearest_nodes(G, end_coords[1], end_coords[0])
        
        try:
            # Calcular a rota mais curta entre os nós usando Dijkstra
            route = nx.shortest_path(G, start_node, end_node, weight='length')
            route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
            rota_coords.extend(route_coords)
        except nx.NetworkXNoPath:
            print(f"Não foi possível encontrar um caminho entre {start} e {end}")
    
    # Adicionar a rota calculada ao mapa
    folium.PolyLine(rota_coords, color='blue', weight=2.5, opacity=1).add_to(mapa)

    # Salvar o mapa em um arquivo HTML
    mapa.save(nome_arquivo)
    print(f"Mapa da rota otimizada foi salvo como: {nome_arquivo}")

# Criar o mapa da rota otimizada e salvar como HTML
output_map_path = 'rota_otimizada_real_mapa.html'
criar_mapa(positions_dict, rota_pontos, output_map_path)

# Não exibe corretamente
#folium.Map(location=map_center, zoom_start=12)


c:\Users\neumann\AppData\Local\Programs\Python\Python310\lib\site-packages\osmnx\graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


Mapa da rota otimizada foi salvo como: rota_otimizada_real_mapa.html
